# Environmental Impact of Agricultural Practices in the World

**ADA Project Milestone 2**

This notebook consists of our initial Data Analysis of the FAOSTAT dataset on Food an agriculture. We will first study the contents of the data and its strucuture, before restructuring it in order to start our analysis. Also, some research questions initially asked will be answered by the end of this notebook.

## A. Initial Analysis

The dataset initially contained 78 csv files, but some of them were discarded as they will not be useful for our analysis. We have selected 25 CSVs that would help us with our analysis.

In [1]:
from glob import glob
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
csv_files = glob('data/**/*.csv')
len(csv_files)

25

We split those 25 csv into different directories, one for each group of csv. Each group corresponds to one category:
```.
+-- data/
|   +-- emissions_agriculture/
|      +-- ...
|   +-- emissions_land/
|      +-- ...
|   +-- environment/
|      +-- ...
|   +-- inputs/
|      +-- ...
|   +-- population/
|      +-- ...
|   +-- production/
|      +-- ...
```

---
### A.1 Schema consistency
---

We will first study the schemas of all the csv files we have in order to see if they are consistent or require changes

#### 1. Checking column names across whole dataset
---

Now, let's scan all the csv files and check their schemas.

In [3]:
from data_processing import scan_columns
all_columns = scan_columns(csv_files)
print("The found columns, grouped, are:\n")
for cols, f in all_columns:
    print(list(sorted(cols)), f"Num files {len(f)}")

The found columns, grouped, are:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 17
['country', 'countrycode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 2
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year', 'yearcode'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'note', 'unit', 'value', 'year', 'yearcode'] Num files 4
['country', 'countrycode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 1


As we can see, sometimes the columns `area` and `areacode` are named `country` and `countrycode`, only because some csv files only contain country data, without country groups. We will rename those as to have a unified schema. Also, some files have the `note`, `elementgroup` and `months` columns. We will look into those in subsequent steps as we are now simply checking whether column naming is consistent.

In order to obtain a more consistent column naming, we will rename `country` to `area` and `countrycode` to `areacode`.

In [4]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}

In [5]:
all_columns_2 = scan_columns(csv_files, column_rename)
print(f"After renaming, we obtain the following columns:\n")
for cols, f in all_columns_2:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year', 'yearcode'] Num files 18
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 2
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year', 'yearcode'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'note', 'unit', 'value', 'year', 'yearcode'] Num files 4


#### 2. Checking which columns to drop
---

1. Column labeled "`note`"

Now we have a few files that have different schemas. One column that we should look into before continuing is `note`, as it is in 4 files.

In [6]:
from data_processing import get_column_unique_values

files_with_note = all_columns_2[-1][1]
note_values = get_column_unique_values(files_with_note, column_rename, None,['note'])
note_values

,note
0,NaN


As we can see, all values for this column are NaN, so we can safely drop the column.

2. Columns "`yearcode`" and "`elementgroup`"

We figured it would be useful to scan for duplicate columns in each dataframe (i.e. columns with different names but same values)

In [7]:
from data_processing import scan_column_duplicates
duplicates = scan_column_duplicates(csv_files, column_rename)
for c, f in duplicates:
    print(f"Duplicates for {c} in {len(f)} files")

Duplicates for [('yearcode', 'year')] in 23 files
Duplicates for [('elementgroup', 'elementcode')] in 1 files


As we can see, most files have `year` and `yearcode` columns which are equal. Hence, we can safely drop this column. However, for `elementgroup` and `elementcode`, they are equal in almost all CSV where they appear (1/2), but not all, so we cannot safely drop it without checking. We choose to keep `elementcode` when those two are equal, and keep them both when they are not.

Hence, we can define a list of columns to be checked and dropped; We drop them if they fulfill any of the following conditions:
 - NaN in all rows
 - Duplicate with another column

In [8]:
check_columns = ["note", "yearcode", "elementgroup"]
all_columns_3 = scan_columns(csv_files, column_rename, check_columns)
print(f"After renaming and dropping columns, we obtain the following columns:\n")
for cols, f in all_columns_3:
    print(list(sorted(cols)), f"Num files {len(f)}")

After renaming and dropping columns, we obtain the following columns:

['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 23
['area', 'areacode', 'element', 'elementcode', 'elementgroup', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'] Num files 1
['area', 'areacode', 'element', 'elementcode', 'flag', 'months', 'monthscode', 'unit', 'value', 'year'] Num files 1


We now have 41 files with identical schemas, and 2 files that have a different one:
 - The file containing the `elementgroup` additional column, which refers to groups of `elements`
 - The file with monthly data and no `item` and `itemcode` columns
 
To obtain the desired format, we can now call `load_dataframe(<file>, column_rename, drop_columns)` with `column_rename = {'country': 'area', 'countrycode': 'areacode'}` and `drop_columns = ["note", "yearcode", "elementgroup"]`

The only remaining csv with different schema are:
- `data/environment/Environment_Temperature_change_E_All_Data_(Normalized).csv` (which contains monthly data)
- `data/environment/Environment_Energy_E_All_Data.csv`, which contains `elementgroup` column

---
### A.2 Schema description

---
Now that we have a unified schema for (almost) all csv files, we can start looking into the meaning of each column and their possible value. 

#### 1. Area columns
---
We will first look into the columns `area` and `areacode`. According to FAOSTAT's website, each area is defined by a unique areacode, however some areas include other ones, i.e. there are grouped areas in the datasets. We would expect a one-to-one mapping between those two columns. Let's see how this looks like

To verify that it is indeed a one-to-one mapping, we will append all values from all csv files, and drop duplicates. Then we group by area and see if the length of the group is 1.

In [9]:
from utils import is_unique_mapping

area_values = get_column_unique_values(csv_files, column_rename, check_columns, ['area', 'areacode'])
is_unique_mapping(area_values, 'area', ['areacode'])

False

It seems that the mapping is not one-to-one. Let's look into why that is. First we group by `areacode` and aggregate to a list of `area` and then we do the inverse. This way, we can see how the mapping is.

In [10]:
grpd = area_values.groupby('area')['areacode'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

Series([], Name: areacode, dtype: object)

The mapping `area` -> `areacode` is unique, now for the opposite:

In [11]:
grpd = area_values.groupby('areacode')['area'].apply(list)
grpd[grpd.apply(lambda x: len(x) > 1)]

areacode
154     [The former Yugoslav Republic of Macedonia, No...
167                             [Czechia, Czech Republic]
209                                 [Eswatini, Swaziland]
299           [Occupied Palestinian Territory, Palestine]
5000                             [World, World + (Total)]
5100                           [Africa, Africa + (Total)]
5101           [Eastern Africa, Eastern Africa + (Total)]
5102             [Middle Africa, Middle Africa + (Total)]
5103         [Northern Africa, Northern Africa + (Total)]
5104         [Southern Africa, Southern Africa + (Total)]
5105           [Western Africa, Western Africa + (Total)]
5200                       [Americas, Americas + (Total)]
5203       [Northern America, Northern America + (Total)]
5204         [Central America, Central America + (Total)]
5206                     [Caribbean, Caribbean + (Total)]
5207             [South America, South America + (Total)]
5300                               [Asia, Asia + (Total)]
5301 

It seems that the same `areacode` is sometimes mapped to multiple `area`, most of the time by adding "+ (Total)". For that, we could drop the `area` column and keep working with `areacode`.

However, we still need a fixed database that maps the `areacode` to `area`, and for this, FAOSTAT provides one CSV file containing the mapping, as well as another one containing a description of country groups (i.e. "Asia") and which contries they contain.

In [12]:
from data_processing import load_dataframe
area_codes = load_dataframe('data/countries.csv')
area_codes.head()

,countrycode,country,m49code,iso2code,iso3code,startyear,endyear
0,2,Afghanistan,4.0,AF,AFG,NaN,NaN
1,5100,Africa,2.0,NaN,X06,NaN,NaN
2,284,Ã land Islands,248.0,NaN,ALA,NaN,NaN
3,3,Albania,8.0,AL,ALB,NaN,NaN
4,4,Algeria,12.0,DZ,DZA,NaN,NaN


In [13]:
country_groups = load_dataframe('data/country_groups.csv')
country_groups.head()

,countrygroupcode,countrygroup,countrycode,country,m49code,iso2code,iso3code
0,5100,Africa,4,Algeria,12.0,DZ,DZA
1,5100,Africa,7,Angola,24.0,AO,AGO
2,5100,Africa,53,Benin,204.0,BJ,BEN
3,5100,Africa,20,Botswana,72.0,BW,BWA
4,5100,Africa,24,British Indian Ocean Territory,86.0,IO,IOT


Here, countries are grouped into multiple `countrygroup`, so we know exactly of which countries each group is formed. These country groups are present in the dataset as `area`, meaning there are aggregated values in the dataset. For example: we can find the emissions for "Algeria" and for "Africa", where the latter is an aggregated value over the whole group. We will need to be careful when aggregating values in the future, as we could account multiple times for one country.

In [14]:
is_unique_mapping(area_codes[['country', 'countrycode']].drop_duplicates(), 'country', ['countrycode'])

True

In [15]:
all(x in area_codes.countrycode.unique() for x in area_values.areacode.unique())

True

Hence, we can use the dataframe `area_codes` to obtain the country's name from it's code, and `country_groups` to get which countries form a group. For this, we can now drop the `area` column in our dataset.

#### 2. Element Columns
---

The `element` and `elementcode` represent the measure quantity for a given `item`. A quantity has a name and a unit, which is why we believe these two columns should also have a one-to-one mapping accross the whole dataset. Also, since an `elementcode` potentially uniquely identifies (`element`, `unit`) pair, we might drop those two columns as to make the csv files smaller and easier to manipulate.

First let's check if indeed this mapping is one-to-one:

In [16]:
element_values = get_column_unique_values(csv_files, column_rename, check_columns, ['elementcode', 'element', 'unit'])
is_unique_mapping(element_values, 'elementcode', ['element', 'unit'])

True

As we can see, `elementcode` uniquely identify (`element`, `unit`) pairs, so we can safely drop those two columns and only use `elementcode`. We will later pivot each csv as to obtain all the `elementcode`s as columns, so we can reduce de number of rows significantly. A mapping using a dictionnary will of course be necessary in order to have a nice GUI where users can select the (element, unit) pair instead of the code.

#### 3. Item columns
---

According to FAOSTAT, the `item` and `itemcode` columns represent item on which measurements were done. For example an item can be `cattle` and the measurement can be "CH4 emissions in gigagrams". 
Similarly to what we did above, we expect `item` and `itemcode` to have a one-to-one relationship. Let's verify this using the same functions

In [17]:
item_values = get_column_unique_values(csv_files, column_rename, check_columns, ['item', 'itemcode'], with_file=True)
is_unique_mapping(item_values[['item', 'itemcode']], 'item', ['itemcode'])

False

It seems that `item` to `itemcode` is not unique for a few items, let's check those and try to understand why it is the case.

In [18]:
grpd = item_values.groupby('itemcode')['item'].agg(set)
grpd[grpd.apply(len) > 1]

Series([], Name: item, dtype: object)

The mapping `itemcode` -> `item` is unique, now let's check the other way around

In [20]:
grouped = item_values.groupby('item')['itemcode'].agg(set)
grouped[grouped.apply(len) >1]

item
Ammonium nitrate (AN)                            {1362, 4003}
Ammonium sulphate                                {1361, 4002}
Cattle                                            {866, 1757}
Chickens                                         {1057, 1054}
Disinfectants                                    {1358, 1351}
Mineral Oils                                     {1354, 1316}
Other Pesticides nes                             {1355, 1359}
Other nitrogenous fertilizers, n.e.c.            {4008, 1369}
Other potassic fertilizers, n.e.c.               {4018, 1391}
Plant Growth Regulators                          {1356, 1341}
Potassium sulphate (sulphate of potash) (SOP)    {4017, 1387}
Urea                                             {4001, 1367}
Name: itemcode, dtype: object

Some items seem to have multiple different item codes, which doesn't seem very normal. 

**a.** It seems that some of those items correspond to nutrients provided throught fertilizers. Let's see in which files those appear. The items related to nutrients are the following :

In [21]:
nutrient_items = ["Ammonium nitrate (AN)", "Ammonium sulphate", "Other nitrogenous fertilizers, n.e.c.", "Other potassic fertilizers, n.e.c.", "Potassium sulphate (sulphate of potash) (SOP)", "Urea"]
item_values[item_values.item.isin(nutrient_items)].file.unique()

array(['data/inputs/Inputs_FertilizersArchive_E_All_Data_(Normalized).csv',
       'data/inputs/Inputs_FertilizersProduct_E_All_Data_(Normalized).csv'],
      dtype=object)

Those items are only present in two dataframes, related to Ferilizer use. If we look closely at the names of csv files, we can see that one of them is "Archive", 
while the other is product. After having looking and reading the documentation for this CSV http://www.fao.org/faostat/en/#data/RA, we understand that it is an archive document, that has not been updated since 2002. 

Before that date, all data on Fertilizer was put into one single dataframe (i.e. info about production, trade and consumption) both in total nutrients and amount of product. After 2002, it was split into nutrient and product, which is why we have two `itemcode` values for those `items`. 

Hence, we should be careful when studying the fertilizer use, as data collection has changed in 2002.


**b.** Some items are related to pesticides. Let's in which csv files they appear

In [22]:
pesticide_items = ["Disinfectants", "Mineral Oils", "Other Pesticides nes", "Plant Growth Regulators"]
item_values[item_values.item.isin(pesticide_items)].file.unique()

array(['data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv'],
      dtype=object)

Some duplicate items with the same name but different codes appear in only one csv file, which seems quite odd. We suspect having duplicate rows in that case. Let's check that out.

In [23]:
from data_processing import check_duplicate_items
t = load_dataframe('data/inputs/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv', column_rename, check_columns)
check_duplicate_items(t, pesticide_items)

Duplicate item for Disinfectants codes [1358 1351]
Duplicate item for Mineral Oils codes [1354 1316]
Duplicate item for Other Pesticides nes codes [1359 1355]
Duplicate item for Plant Growth Regulators codes [1356 1341]


We now know that there are potentially duplicate rows for the same `item` but a different `itemcode` (with the same measurements). Hence, we need to add a functionnality that checks for duplicated items in the dataframe.

**c.** Now let's look at livestock items

In [24]:
livestock_items = ["Cattle", "Chickens"]
for i in item_values[item_values.item.isin(livestock_items)].groupby(['item', 'itemcode']).agg(set).reset_index().values:
    print(i)

['Cattle' 866
 {'data/production/Production_Livestock_E_All_Data_(Normalized).csv'}]
['Cattle' 1757
 {'data/emissions_agriculture/Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).csv', 'data/emissions_agriculture/Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).csv', 'data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).csv', 'data/emissions_agriculture/Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).csv'}]
['Chickens' 1054
 {'data/emissions_agriculture/Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).csv', 'data/emissions_agriculture/Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).csv', 'data/emissions_agriculture/Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).csv'}]
['Chickens' 1057
 {'data/production/Production_Livestock_E_All_Data_(Normalized).csv'}]


The `itemcode` -> `item` seems one-to-one in each csv file, but we see that there are two distinct `itemcode` for "Cattle" and "Chickens" accross all csv files.

It seems that different values for `itemcode` for both "Chickens" and "Cattles" differ when the measured quantity is realted to emissions. Thus, this might be a discrepancy in the data as it does not appear for other items. We will treat this discrepancy once all csv have been merged into one.

#### 4. Flag Columns
---

According to FAOSTAT, not all the data points are official figures reported by individual countries. Some are estimates, others are aggregates done by external entities. In order to understand the meaning of those flagas, they provide a csv file containing the description of each.

In [25]:
flags = load_dataframe("data/flags.csv")
flags.head()

,flag,flags
0,NaN,Official data
1,*,Unofficial figure
2,A,"Aggregate, may include official, semi-official..."
3,B,Balance
4,Bk,Break in series


In [26]:
print(f"There are {len(flags)} flags")

There are 41 flags


---
### A.3 Conclusion about Schemas

---

After having applied some analysis on the initially different schemas of the dataframes, we are now able to obtain identitcal schemas for 41 of them, and left two of them where we believe they should not be changed as some information could be lost.

The common schema has the following columns:

`['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year']`

For now, we have come up with a few steps that should be applied when loading a dataframe:
 - Rename columns to remove spaces and captial letters
 - Rename "Country" to "Area" and "CountryCode" to "Areacode"
 - Drop columns that are either duplicates or NaN values
 - Rename "gigagrams" to "Gigagrams", as otherwise we would have a non one-to-one mapping for `elementcode`
 - Check for items if they have duplicate rows.

The function that does the first 4 cleaning steps is `data_processing/load_dataframe`, and the one that puts everything together is `data_processing/load_clean_dataframe`.

Also, we have decided to only work with codes (i.e. `areacode` and `elementcode`) and drop `area`, `element` and `unit`. Of course, we will need a mapping from `code` -> `value`. Additionally, in order to keep track of `countrygroup` and do know how values are aggregated, we will also create a mapping that tells us which countries form a group, by codes.

Hence, to load a dataframe, we simply call :
```
df = load_clean_dataframe(<filename>, column_rename, check_columns, drop_columns)
```

with  `column_rename = {'country': 'area', 'countrycode': 'areacode'}` and `check_columns = ["note", "yearcode", "elementgroup"]` and `drop_columns = ["area", "item", "element", "unit"]`

In [36]:
column_rename = {'country': 'area', 'countrycode': 'areacode'}
check_columns = ["note", "yearcode", "elementgroup"]
drop_columns = ["area", "item", "element", "unit"]

Now we need to get the mappings for :
- `elementcode` -> (`element`, `unit`) (unique in both ways)
- `itemcode` -> `item` (unique in that way, but not the other way around)
- `areacode` -> `area` (unique in both ways)

In [37]:
from mappings import get_mapping, get_area_mapping, get_country_groups
item_mapping = get_mapping(csv_files, column_rename, check_columns, ['itemcode','item'])
element_mapping = get_mapping(csv_files, column_rename, check_columns, ['elementcode', 'element', 'unit'])


In [38]:
from mappings import get_area_mapping, get_country_groups
area_mapping = get_area_mapping("data/countries.csv")
country_groups = get_country_groups("data/country_groups.csv")

---
---
## B. Reshaping the Database

For now everything was stored in database format (i.e. Row format), but we would like to have elements in columns, so that we can reduce drastically the number of rows, and so we can have all the measurements for a single (`itemcode`, `areacode`, `year`) pair.

---
### B.1 Merging all files with same schema

---

Before reshaping, we know that 41 csv files have the same schema. Thus, it would be good to have them all in a single dataframe in order to facilitate access to values. We will then create a single dataframe from all those 41 (over 43 in total). This dataframe will then have potentially duplicate rows, which we will drop.

In [39]:
from data_processing import load_clean_dataframe
def load_all_df_with_schema(files, schema):
    dfs = []
    for f in csv_files:
        df = load_clean_dataframe(f, column_rename, check_columns, drop_columns) # Load the DF
        if all(x in schema for x in df.columns):  # Check if schema corresponds
            df = df.assign(file=f)
            dfs.append(df)
    df = pd.concat(dfs).reset_index(drop=True)
    shape = df.shape[0]
    
    to_keep = df.drop(columns=['file']).drop_duplicates().index # Drop all duplicated row, after removing `file`
    df = df.loc[to_keep].reset_index(drop=True)
    print(f"Dropped {shape - df.shape[0]} duplicate rows")
    return df

In [61]:
df = load_all_df_with_schema(csv_files, ['area', 'areacode', 'element', 'elementcode', 'flag', 'item', 'itemcode', 'unit', 'value', 'year'])

Dropped duplicate item Disinfectants with codes [1358, 1351] (Dropped 1351)
Dropped duplicate item Mineral Oils with codes [1354, 1316] (Dropped 1316)
Dropped duplicate item Other Pesticides nes with codes [1359, 1355] (Dropped 1355)
Dropped duplicate item Plant Growth Regulators with codes [1356, 1341] (Dropped 1341)
Dropped 725925 duplicate rows


In [62]:
import gc
gc.collect()

1336

---
### B.2 Checking for duplicate measurements

---

Ideally, we would like to have a single row for each (`areacode`, `itemcode`, `elementcode` and `year`), i.e one measurement. Let's check if it is the case:

In [63]:
grouped = df.groupby(['areacode', 'itemcode', 'elementcode', 'year'])[['value', 'flag']].nunique(dropna=False).reset_index()
grouped['value'].value_counts(dropna=False) / grouped.shape[0]

1    0.998824
2    0.001176
Name: value, dtype: float64

As we can see, it is not the case:
    - Most of the time there is one measurement (~99.9%)
    - Some have two measurements (~0.1%)
This could be due to different `value` and/or different `flag`

Let's look at those values

#### 1. Assessing Elements that have duplicates

In order to remediate to this issue, we will look at the differences between the values, when there are multiple ones, and group by `elementcode`

In [66]:
from utils import get_percentage_diff

def get_discrepancies(df):
    indexed = df.set_index(['areacode', 'itemcode', 'elementcode', 'year']) # Set the index to what what should be unique
    duplicated_1 = indexed.index.duplicated(keep='first')  # Take the first duplicate
    duplicated_2 = indexed.index.duplicated(keep='last')  # Take the second one
    joined = indexed.loc[duplicated_1].join(indexed.loc[duplicated_2], rsuffix='_2')  # Join on the index to have both values and indices
    return joined

def get_discrepancies_by(df,by):
    discrepancies_by = df.reset_index().groupby(by)\
                                 .apply(lambda x: pd.Series({'mean_diff': np.mean(x['value'] - x['value_2']), # Mean difference
                                                             'std': np.std(x['value'] - x['value_2']),        # STD of difference
                                                             'num': x.shape[0],                                 # Number of different values
                                                             'mean_perc_diff': np.mean(get_percentage_diff(x['value'], x['value_2'])*100)})) # Average difference in percentage
    return discrepancies_by.sort_values('mean_perc_diff')

In [68]:
joined = get_discrepancies(df)
get_discrepancies_by(joined, ['file', 'file_2']).reset_index()

,file,file_2,mean_diff,std,num,mean_perc_diff
0,data/emissions_agriculture/Emissions_Agricultu...,data/production/Production_Crops_E_All_Data_(N...,2.754141e+06,7.064850e+05,57.0,4.526840
1,data/emissions_agriculture/Emissions_Agricultu...,data/environment/Environment_Emissions_by_Sect...,5.025583e+02,4.838841e+03,14287.0,5.326875
2,data/emissions_agriculture/Emissions_Agricultu...,data/production/Production_Livestock_E_All_Dat...,-1.117536e+07,3.105129e+07,1153.0,5.874584
3,data/emissions_land/Emissions_Land_Use_Land_Us...,data/environment/Environment_Emissions_by_Sect...,2.293806e+03,9.149522e+03,5177.0,88.779951


In the above DataFrame, we have the `mean_diff` (mean difference between values), `std` and `mean_perc_diff` (mean percentage difference), grouped by files.
Hence, each line gives us a description of how much the same measurements differ on average.

It seems that the csv file `data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv` introduces a lot of discrepancies, which might be due to different techniques of estimating measurements. We will drop this csv from the overall dataframe, but will maybe use it later.


In [69]:
df = df[df.file != "data/environment/Environment_Emissions_by_Sector_E_All_Data_(Normalized).csv"]
joined = get_discrepancies(df)

Now that we have dropped the potentially problematic csv, we will print the same statistics, but grouped by areacode.

In [87]:
by_area = get_discrepancies_by(joined, 'areacode')#.sort_values(['num', 'mean_perc_diff'])
by_area.index = [area_mapping[i] for i in by_area.index]
by_area[by_area['std'] > 0].sort_values('std')

,mean_diff,std,num,mean_perc_diff
Northern Africa,8.000000e-01,6.000000e-01,10.0,2.542450e-05
Asia,-1.117647e+00,6.758309e-01,17.0,7.403882e-06
Western Asia,-6.666667e-01,7.453560e-01,6.0,9.504502e-05
Southern Africa,6.000000e-01,8.000000e-01,5.0,1.592143e-03
Oceania,-6.000000e-01,8.000000e-01,10.0,3.794671e-02
Southern Europe,-5.000000e-01,8.660254e-01,4.0,4.193061e-04
Europe,-5.000000e-01,8.660254e-01,8.0,2.039936e-04
Eastern Asia,-3.333333e-01,9.428090e-01,3.0,4.623518e-07
China,-3.333333e-01,9.428090e-01,6.0,3.446486e-06
Polynesia,-3.333333e-01,9.428090e-01,6.0,5.009245e-03


It seems that problematic areas are in area groups. Since we already know how area groups are formed, we can drop all values that are associated to a group and recompute them using `country_groups`

In [88]:
group_codes = set(country_groups.keys())
df = df[~df.areacode.isin(group_codes)]  # Drop all country groups

In [92]:
joined = get_discrepancies(df)
(joined.value - joined.value_2).describe()

count    70.000000
mean     -0.057143
std       1.005575
min      -1.000000
25%      -1.000000
50%      -1.000000
75%       1.000000
max       1.000000
dtype: float64

As we can see, now most differences have a very small mean and standard deviation. Also, there are only 70 duplicate measurements. In order to have unique ones, we will take the average of the two values

#### 2. Taking care of duplicate measurements

Now that we have understood where the big differences in duplicates appear, we focus on those that are still here, but are quite small. In order to get rid of duplicate measurements, we will work with a combination of the `value` and `flag` columns.

In [94]:
def is_close(value_1, value_2, threshold=0.01):
    diff = get_percentage_diff(value_1, value_2)
    return (value_1 == value_2) | (diff < threshold) | (value_1.isna() & value_2.isna())

def is_same_flag(flag_1, flag_2):
    return (flag_1 == flag_2) | (flag_1.isna() & flag_2.isna())

In [95]:
if all(is_close(joined['value'], joined['value_2'])):
    print("All duplicated measruements' difference are at less than 1%")

All duplicated measruements' difference are at less than 1%


In [96]:
same_flag = joined[is_same_flag(joined.flag, joined.flag_2)]
print("{:.2f}% of duplicated rows have the same flag".format(same_flag.shape[0]/ joined.shape[0]*100))

100.00% of duplicated rows have the same flag


Since most duplicated measurements'values are really close, we will take the mean if there are two different values.

In [97]:
df = df.drop(columns=['file', 'flag']).drop_duplicates().groupby(['areacode', 'itemcode', 'elementcode', 'year'])['value'].mean().reset_index()

---
### B.3 Reshaping measurements as columns
---

Now that we have unique measurements, we will reshape the dataframe to put each measurement for an item as column, so that for each (`areacode`, `itemcode`, `year`) we have all the measurements as columns.

In [98]:
reshaped_df = df.set_index(['areacode', 'itemcode', 'elementcode', 'year'])['value'].unstack(level=2).reset_index()

1. Treating the Cattle (codes 866 and 1757) and Chicken (codes 1054 and 1057) duplicate items 

In [102]:
cattle_1 = get_item(reshaped_df, 866)
cattle_2 = get_item(reshaped_df, 1757)

In [133]:
cattle_1.head()

elementcode,areacode,itemcode,year,5111
1080,1,866,1992,566452.0
1081,1,866,1993,498863.0
1082,1,866,1994,501635.0
1083,1,866,1995,503693.0
1084,1,866,1996,507512.0


In [134]:
cattle_2.head()

elementcode,areacode,itemcode,year,5111,72254,72256,72300,72301,72306,72310,...,72381,72386,72436,72446,723601,723602,723801,723802,723811,723812
3138,1,1757,1992,566452.0,21.3271,0.8176,1.1196,0.0303,0.0155,347.0656,...,1.351598e+06,1.668742e+06,4.7979,17.1693,0.0923,0.1039,5.875888e+06,8.813832e+06,270319.6366,405479.4549
3139,1,1757,1993,498863.0,19.3270,0.7563,1.0116,0.0268,0.0137,313.6046,...,1.197811e+06,1.506630e+06,4.2624,15.8833,0.0834,0.0939,5.309387e+06,7.964081e+06,239562.2715,359343.4073
3140,1,1757,1994,501635.0,19.5984,0.7715,1.0250,0.0270,0.0139,317.7398,...,1.206720e+06,1.526136e+06,4.2972,16.2011,0.0845,0.0951,5.379397e+06,8.069095e+06,241343.9611,362015.9417
3141,1,1757,1995,503693.0,19.7545,0.7797,1.0327,0.0272,0.0139,320.1486,...,1.212711e+06,1.537541e+06,4.3199,16.3736,0.0852,0.0958,5.420177e+06,8.130266e+06,242542.2022,363813.3032
3142,1,1757,1996,507512.0,19.8849,0.7843,1.0397,0.0274,0.0140,322.2929,...,1.221639e+06,1.547881e+06,4.3514,16.4706,0.0857,0.0965,5.456482e+06,8.184722e+06,244327.8668,366491.8001


It seems that `cattle` with itemcode `866` is only associated to one `elementcode` (5111), while the other itemcode (1757) has much more. In order to see if we should drop the item code `1757`, let's check if all measurements for both item codes are equal.

In [154]:
from utils import all_present_values_equal
dd1 = cattle_1.drop(columns=['itemcode']).set_index(['areacode', 'year'])
dd2 = cattle_2[["areacode", "year", 5111]].set_index(['areacode', 'year'])
all_present_values_equal(dd1, dd2)

All present items are equal
Some measurements exist in the first but not the second


Hence, we can replace the itemcode `866` by `1757`, and re-do the reshaping so that we keep the values of `866` that don't exist for `1757`, and then drop all duplicate rows.

In [149]:
df = df.assign(itemcode=df.itemcode.apply(lambda x: x if x != 866 else 1757))
df = df.drop_duplicates()

In [150]:
chickens_1 = get_item(reshaped_df, 1054)
chickens_2 = get_item(reshaped_df, 1057)

In [155]:
chickens_1.head()

elementcode,areacode,itemcode,year,5111,72256,72300,72301,72306,72310,72311,...,72381,72386,72436,72446,723601,723602,723801,723802,723811,723812
1770,1,1054,1992,8934000.0,0.0893,0.0979,0.0235,0.0082,30.3502,7.2846,...,1.049382e+06,1.049382e+06,2.5560,1.8761,0.0081,0.0091,513835.3638,770753.0457,209876.4162,314814.6243
1771,1,1054,1993,2655000.0,0.0266,0.0351,0.0084,0.0030,10.8736,2.6099,...,3.759633e+05,3.759633e+05,0.9157,0.5576,0.0029,0.0033,184092.3855,276138.5783,75192.6645,112788.9968
1772,1,1054,1994,2686000.0,0.0269,0.0354,0.0085,0.0030,10.9676,2.6324,...,3.792119e+05,3.792119e+05,0.9237,0.5641,0.0029,0.0033,185683.0482,278524.5723,75842.3718,113763.5577
1773,1,1054,1995,2713000.0,0.0271,0.0356,0.0086,0.0030,11.0494,2.6520,...,3.820412e+05,3.820412e+05,0.9305,0.5697,0.0029,0.0033,187068.4641,280602.6961,76408.2459,114612.3689
1774,1,1054,1996,2721000.0,0.0272,0.0357,0.0086,0.0030,11.0736,2.6579,...,3.828796e+05,3.828796e+05,0.9326,0.5714,0.0029,0.0033,187478.9577,281218.4365,76575.9123,114863.8685


In [156]:
chickens_2.head()

elementcode,areacode,itemcode,year,5112
1820,1,1057,1992,8934.0
1821,1,1057,1993,2655.0
1822,1,1057,1994,2686.0
1823,1,1057,1995,2713.0
1824,1,1057,1996,2721.0


In [163]:
element_mapping[5112], element_mapping[5111]

(('Stocks', '1000 Head'), ('Stocks', 'Head'))

It seems that `chickens` with itemcode `1057` is only associated to one `elementcode` (5112) i.e. Stocks in 1000 Head, while the other itemcode (1054) has elementcode `5111` which is Stocks in Head, as well as other measurements. Hence, we decide to drop itemcode 1057.

In [164]:
df = df[df.itemcode != 1057]
df = df.drop_duplicates()

2. Final Reshaping and saving dataframe to file

In [165]:
reshaped_df = df.set_index(['areacode', 'itemcode', 'elementcode', 'year'])['value'].unstack(level=2).reset_index()

In [166]:
reshaped_df.to_pickle("reshaped_df.pkl")

In [169]:
reshaped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820426 entries, 0 to 1820425
Columns: 135 entries, areacode to 724410
dtypes: float64(132), int64(3)
memory usage: 1.8 GB


The overall dataframe is 1.8 GB, so it easily fits in memory

In [170]:
reshaped_df.describe()

elementcode,areacode,itemcode,year,511,512,513,551,561,5026,5111,...,723601,723602,723631,723632,723801,723802,723811,723812,724310,724410
count,1.820426e+06,1.820426e+06,1.820426e+06,3.434900e+04,29582.000000,29582.000000,22419.000000,2.241900e+04,9.286000e+03,2.135540e+05,...,203764.000000,203764.000000,10430.000000,10430.000000,2.037640e+05,2.037640e+05,2.037640e+05,2.037640e+05,43106.000000,43106.000000
mean,1.271487e+02,1.751123e+03,1.991963e+03,3.862612e+04,22587.315000,22257.645266,15938.502169,1.700318e+04,1.551183e+05,1.621639e+07,...,0.197304,0.221967,0.766110,1.723747,1.255575e+07,1.883363e+07,4.976118e+06,7.464177e+06,82.069813,28.434541
std,7.580728e+01,1.754910e+03,1.842096e+01,1.495703e+05,81963.956597,78356.101142,77669.105095,7.422375e+04,4.280476e+05,1.432449e+08,...,1.084348,1.219892,3.783892,8.513758,6.900399e+07,1.035060e+08,3.261376e+07,4.892064e+07,407.025155,244.364555
min,1.000000e+00,1.500000e+01,1.950000e+03,6.420000e-01,6.812000,6.889000,0.000000,0.000000e+00,8.604000e-01,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-365.101000,-3.588800
25%,6.000000e+01,6.030000e+02,1.977000e+03,3.408740e+02,627.016750,631.188250,73.752000,1.240245e+02,2.331951e+03,2.209250e+04,...,0.000000,0.000100,0.001600,0.003500,2.964533e+03,4.446800e+03,4.144672e+03,6.217007e+03,0.000000,0.000000
50%,1.230000e+02,1.137000e+03,1.993000e+03,4.869609e+03,3598.241000,3591.666000,1305.746000,1.792539e+03,1.608944e+04,3.528000e+05,...,0.002300,0.002500,0.026500,0.059650,1.441363e+05,2.162044e+05,6.997996e+04,1.049699e+05,0.030500,0.048600
75%,1.890000e+02,1.765000e+03,2.005000e+03,2.165797e+04,14044.854250,14247.120000,7574.802500,8.159051e+03,1.036142e+05,3.500000e+06,...,0.039200,0.044100,0.258175,0.580875,2.494983e+06,3.742475e+06,8.790296e+05,1.318544e+06,3.812400,1.570900
max,3.510000e+02,6.813000e+03,2.100000e+03,1.651843e+06,847170.144000,805429.477000,909426.803000,1.122105e+06,4.775398e+06,8.759451e+09,...,35.531700,39.973100,49.102500,110.480500,2.261107e+09,3.391660e+09,1.479223e+09,2.218834e+09,6600.373500,5616.697600


In [171]:
reshaped_df

elementcode,areacode,itemcode,year,511,512,513,551,561,5026,5111,...,723601,723602,723631,723632,723801,723802,723811,723812,724310,724410
0,1,15,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,15,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,15,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,15,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,15,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1820421,351,6813,2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4592.5074,4870.9685
1820422,351,6813,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4746.8109,5107.8953
1820423,351,6813,2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5046.5824,5413.6568
1820424,351,6813,2011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5312.5243,5594.4166


In [188]:
reshaped_df = reshaped_df[reshaped_df.year.apply(lambda x: x < 2020)]

In [198]:
t = get_item(reshaped_df, [15]).drop('itemcode', axis=1)

In [201]:
t = t.set_index(['areacode', 'year'])

In [202]:
for i in t.index:
    if t.loc[i]

elementcode         5312     5419         5510          7245   72247    72257  \
areacode year                                                                   
1        1992     65500.0  21600.0     141483.0  2.620000e+04    2.7   0.0707   
         1993     97900.0  22257.0     217900.0  3.916000e+04    2.7   0.1057   
         1994     85697.0  17842.0     152900.0  3.427880e+04    2.7   0.0926   
         1995     60362.0  25480.0     153800.0  2.414480e+04    2.7   0.0652   
         1996     85450.0  23569.0     201400.0  3.418000e+04    2.7   0.0923   
...                   ...      ...          ...           ...    ...      ...   
351      2013  24119335.0  50553.0  121930527.0  9.647734e+06    2.7  26.0489   
         2014  24071629.0  52433.0  126215211.0  9.628652e+06    2.7  25.9974   
         2015  24599116.0  53923.0  132646335.0  9.839646e+06    2.7  26.5670   
         2016  24698641.0  53962.0  133278338.0  9.879456e+06    2.7  26.6745   
         2017  24510393.0  54810.0  134340630.0  9.804157e+06    2.7  26.4712   

elementcode     72292   72297    72302   72307      72312     72317    72342  \
areacode year                                                                  
1        1992  0.0122  0.0001   0.0368  0.0018    11.4118    2.0541   0.0301   
         1993  0.0123  0.0001   0.0565  0.0027    17.5068    3.0701   0.0461   
         1994  0.0123  0.0001   0.0409  0.0024    12.6762    2.6875   0.0334   
         1995  0.0123  0.0001   0.0392  0.0017    12.1555    1.8930   0.0320   
         1996  0.0123  0.0001   0.0518  0.0024    16.0651    2.6797   0.0423   
...               ...     ...      ...     ...        ...       ...      ...   
351      2013  0.0122  0.0001  29.3218  0.6753  9089.7705  756.3823  23.9362   
         2014  0.0123  0.0001  30.2857  0.6740  9388.5593  754.8863  24.7230   
         2015  0.0123  0.0001  31.7769  0.6888  9850.8328  771.4283  25.9403   
         2016  0.0123  0.0001  31.9270  0.6916  9897.3604  774.5494  26.0628   
         2017  0.0122  0.0001  32.1528  0.6863  9967.3733  768.6459  26.2472   

elementcode        72352   72362      72372         72392     72437     72447  
areacode year                                                                  
1        1992     9.3157  0.0068     2.0960  1.912320e+06    0.5685    1.4855  
         1993    14.2913  0.0104     3.2155  2.933695e+06    0.8498    2.2204  
         1994    10.3480  0.0075     2.3283  2.124213e+06    0.7438    1.9436  
         1995     9.9228  0.0072     2.2326  2.036943e+06    0.5239    1.3690  
         1996    13.1144  0.0095     2.9507  2.692099e+06    0.7417    1.9380  
...                  ...     ...        ...           ...       ...       ...  
351      2013  7420.2208  5.3856  1669.5497  1.523212e+09  209.3558  547.0265  
         2014  7664.1301  5.5627  1724.4293  1.573282e+09  208.9417  545.9445  
         2015  8041.4962  5.8366  1809.3366  1.650747e+09  213.5203  557.9080  
         2016  8079.4779  5.8641  1817.8825  1.658544e+09  214.3842  560.1652  
         2017  8136.6313  5.9056  1830.7420  1.670276e+09  212.7502  555.8957  

[6348 rows x 19 columns]

In [204]:
any(reshaped_df.isna().all())

False

In [208]:
for i in t.index:
    print(t.loc[i])
    break

elementcode
5312     6.550000e+04
5419     2.160000e+04
5510     1.414830e+05
7245     2.620000e+04
72247    2.700000e+00
72257    7.070000e-02
72292    1.220000e-02
72297    1.000000e-04
72302    3.680000e-02
72307    1.800000e-03
72312    1.141180e+01
72317    2.054100e+00
72342    3.010000e-02
72352    9.315700e+00
72362    6.800000e-03
72372    2.096000e+00
72392    1.912320e+06
72437    5.685000e-01
72447    1.485500e+00
Name: (1, 1992), dtype: float64
